In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/datasetnorm1.xlsx")
#data = pd.read_excel("C:/Users/Niku/Documents/dataset/datatest.xlsx")
data.shape

(1994, 121)

In [3]:
Normdata = data
Normdata = np.matrix(Normdata)
for i in range(1, Normdata.shape[1]):
    cur = Normdata[:, i]
    mx = max(cur)*1.0
    mn = min(cur)*1.0
    for j in range(0, len(cur)):
        cur[j] = (cur[j] - mn)/(mx - mn)
    Normdata[:, i] = cur
colus = Normdata.shape[1]
X = Normdata[:,0:colus-1]
y = Normdata[:,colus-1:colus]
# convert to numpy arrays and initalize the parameter array theta
#X = np.array(X.values)
#y = np.array(y.values)

In [4]:
#X = np.matrix(data)
#X = X[:,0:35]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1)
X_train.shape, y_train.shape

((997L, 120L), (997L, 1L))

In [6]:
k = 2
rows = X_train.shape[0]
cols = X_train.shape[1]
cent = np.zeros((k, X_train.shape[1]))

In [7]:
def closest_centroid(X, cent, k):
    min_dist = 100000000
    closest = 0
    for i in range(k):
        dist = X - cent[i]
        dist = np.array(dist)
        dist = np.sum(dist**2)
        if dist < min_dist:
            min_dist = dist
            closest = i
    return closest

In [17]:
def kmeans(X):
    cent = np.zeros((k, X.shape[1]))
    c = np.zeros(rows)
    for i in range(k):
        r = int(np.random.rand()*rows)
        cent[i] = X[r]
    #print cent
    for iter in range(100):
        for i in range(rows):
            c[i] = closest_centroid(X[i, :], cent, k)
            #print c[i]
        cluster0 = X[np.where(c == 0)[0],:]
        cluster1 = X[np.where(c == 1)[0],:]
        for j in range(cols):
            cent[0, j] = np.mean(cluster0[:,j])
        for j in range(cols):
            cent[1, j] = np.mean(cluster1[:,j])
    return c, cent, cluster0, cluster1

In [18]:
y_pred, cent, clus0, clus1 = kmeans(X_train)

In [19]:
row = X_test.shape[0]
y_pred = np.zeros(row)
def predictKmeans(X, cent, k):
    for i in range(row):
        y_pred[i] = closest_centroid(X[i, :], cent, k)
    return y_pred

In [20]:
y_pred = predictKmeans(X_test, cent, k)

In [21]:
y_pred = np.zeros(X_test.shape[0])
count = 0
prec = np.zeros(10)
recal = np.zeros(10)
f1 = np.zeros(10)
acc = np.zeros(10)
auc =  np.zeros(10)
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
for l in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
    y_pred, cent, clus0, clus1 = kmeans(X_train)
    y_pred = predictKmeans(X_test, cent, k)
    prec[l] = precision_score(y_test, y_pred, average="weighted")
    recal[l] = recall_score(y_test, y_pred, average="weighted")
    f1[l] = f1_score(y_test, y_pred, average="weighted")
    acc[l] = accuracy_score(y_test, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred) 
    auc[l] = metrics.auc(fpr, tpr)

In [22]:
np.mean(prec), np.mean(recal), np.mean(f1), np.mean(acc), np.mean(auc)

(0.45674565932330502,
 0.4361083249749248,
 0.43849826820782445,
 0.4361083249749248,
 0.44152450393332482)